# Prediction on Google ML with Keras

### Note
- Ensure the same version is used locally and on Google cloud https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list

## Setup

### Configuration

In [1]:
ROOT_PATH = '../services/prediction-keras'

MODEL_PATH = '%s/models' % (ROOT_PATH)
EXPORTED_MODEL_PATH = '%s/keras_exported_model' % (MODEL_PATH)
EXPORTED_SCALAR_PATH = '%s/scaler.dat' % (MODEL_PATH)

DATA_PATH = '%s/data' % (ROOT_PATH)
DATA_INPUT_CSV_PATH = '%s/proposed_new_product.csv' % (DATA_PATH)
DATA_INPUT_JSON_PATH = '%s/sample_input_prescaled.json' % (DATA_PATH)

BUCKET_NAME = 'keras_earning-prediction-1000'
BUCKET_FOLDER_NAME = 'earnings_v1'
BUCKET_URL = 'gs://%s' % (BUCKET_NAME)
BUCKET_FOLDER_URL = '%s/%s' % (BUCKET_URL, BUCKET_FOLDER_NAME)

MODEL_NAME = 'keras_earnings_prediction'
MODEL_VERSION = 'v1'
MODEL_RUNTIME_VERSION = '1.2'

PROJECT_ID = "gde-core-dev"
REGION = 'us-central1'
CREDENTIALS_FILE = "./credentials.json"

### Libraries

- google-api-python-client
- oauth2client
- pickle

In [2]:
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery

import pickle

### Upload the model to Google Cloud bucket

In [3]:
# Make the bucket
!gsutil mb -l { REGION } { BUCKET_URL }
!gsutil cp -r { EXPORTED_MODEL_PATH } { BUCKET_FOLDER_URL }

Creating gs://keras_earning-prediction-1000/...
Copying file://../services/prediction-keras/models/keras_exported_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://../services/prediction-keras/models/keras_exported_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://../services/prediction-keras/models/keras_exported_model/variables/variables.index [Content-Type=application/octet-stream]...
- [3 files][500.8 KiB/500.8 KiB]                                                
Operation completed over 3 objects/500.8 KiB.                                    


### Create the model with Google ML Engine

In [4]:
!gcloud ml-engine models create { MODEL_NAME } --regions { REGION }

Created ml engine model [projects/gde-core-dev/models/keras_earnings_prediction].


In [5]:
!gcloud ml-engine versions create { MODEL_VERSION } \
    --model={ MODEL_NAME } \
    --origin={ BUCKET_FOLDER_URL }/ \
    --runtime-version={ MODEL_RUNTIME_VERSION }

Creating version (this might take a few minutes)......done.                    


## Prediction

### Input

In [6]:
input = [[0.5, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]]

In [7]:
scaler = pickle.load(open(EXPORTED_SCALAR_PATH, 'rb'))

### Predict with Google ML using gcloud

In [8]:
output = !gcloud ml-engine predict \
    --model={ MODEL_NAME } \
    --json-instances={ DATA_INPUT_JSON_PATH }

print(output)

['EARNINGS', '[0.794971764087677]']


In [9]:
prediction_scaled = eval(output[1])

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = (prediction_scaled[0] - scaler.min_[8]) / scaler.scale_[8]

print("Our neural network predicted earnings of ${}".format(prediction))

Our neural network predicted earnings of $246398.83704453707


### Predict from Google ML Engine using SDK

In [10]:
# These are the values we want a prediction for
# These are the values we want a prediction for
inputs_for_prediction = [{
    "input": [0.5, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]
}]

# Connect to the Google Cloud-ML Service
credentials = GoogleCredentials.from_stream(CREDENTIALS_FILE)
service = googleapiclient.discovery.build('ml', 'v1', credentials=credentials)

# Connect to our Prediction Model
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
response = service.projects().predict(
    name=name,
    body={'instances': inputs_for_prediction}
).execute()

# Report any errors
if 'error' in response:
    raise RuntimeError(response['error'])

# Grab the results from the response object
results = response['predictions']

# Print the results!
print(results)

[{'earnings': [0.794971764087677]}]


## Tear down

### Delete the model from Google ML Engine

In [11]:
!gcloud ml-engine versions delete { MODEL_VERSION } \
    --model={ MODEL_NAME } \
    --quiet
!gcloud ml-engine models delete { MODEL_NAME } --quiet

Deleting version [v1]......done.                                               
Deleting model [keras_earnings_prediction]...done.                             


In [12]:
!gsutil -m rm -r { BUCKET_URL } 

Removing gs://keras_earning-prediction-1000/earnings_v1/saved_model.pb#1540427199434100...
Removing gs://keras_earning-prediction-1000/earnings_v1/variables/variables.data-00000-of-00001#1540427199764949...
Removing gs://keras_earning-prediction-1000/earnings_v1/variables/variables.index#1540427199987240...
/ [3/3 objects] 100% Done                                                       
Operation completed over 3 objects.                                              
Removing gs://keras_earning-prediction-1000/...


## Troubleshooting

In [13]:
# Check the model signature with saved_model_cli

!saved_model_cli show --dir { EXPORTED_MODEL_PATH }  --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 9)
        name: layer_1_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['earnings'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: output_layer_1/BiasAdd:0
  Method name is: tensorflow/serving/predict


In [14]:
# Test prediction with saved_model_cli

!saved_model_cli run \
    --dir { EXPORTED_MODEL_PATH } \
    --tag_set serve \
    --signature_def serving_default \
    --input_exprs 'input={ input }'

2018-10-24 17:28:55.983644: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for output key earnings:
[[0.79497176]]
